<a href="https://colab.research.google.com/github/linamarwabelkilani/modelisation-hydrologique/blob/main/model_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importation des données depuis github et preparation des types pour le model

In [1]:
#import dataset from github
import pandas as pd
data_url = "https://raw.githubusercontent.com/linamarwabelkilani/modelisation-hydrologique/main/Dataset.csv"
df = pd.read_csv(data_url, on_bad_lines='skip',sep=";")


df['Station']=df['Station'].astype(str)
df = df.astype({'Station':'string'})
df = df.astype({'Date':'string'})
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True,errors='coerce')
df = df.dropna()
print(df.dtypes)
df

Date             datetime64[ns]
Station                  string
Precipitation           float64
dtype: object


,Date,Station,Precipitation
0,1951-06-01,AIN BEYA OUED RHEZAL,0.0
1,1951-06-02,AIN BEYA OUED RHEZAL,0.0
2,1951-06-03,AIN BEYA OUED RHEZAL,0.0
3,1951-06-04,AIN BEYA OUED RHEZAL,0.0
4,1951-06-05,AIN BEYA OUED RHEZAL,0.0
...,...,...,...
2103688,2015-08-27,ZOUARINE GARE,3.0
2103689,2015-08-28,ZOUARINE GARE,7.0
2103690,2015-08-29,ZOUARINE GARE,0.0
2103691,2015-08-30,ZOUARINE GARE,0.0


preparation de model 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow_probability as tfp

#getting data
data=df[:100000]

#adding cumbel distributions
dist = tfp.distributions.Gumbel(loc=0., scale=1.)
data["Precipitation"]=dist.prob(data["Precipitation"])


#creating model
model = tf.keras.Sequential([keras.layers.Dense(units=2, input_shape=(1,)),
                             keras.layers.Activation('relu'),
                             keras.layers.Dense(units=1, input_shape=(1,)),
])


x=data["Date"]
y=data["Precipitation"]
x_train = np.asarray(x).astype(np.float32)
y_train = np.asarray(y).astype(np.float32)

model.compile(optimizer='rmsprop', loss='kl_divergence',)

#train the model
model.fit(x_train, y_train, epochs=10,batch_size=32)




Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [47]:
#Using Model
data_test=data=df[100000:100010]
x_test = np.asarray(data_test["Date"]).astype(np.float32)
y_test = np.asarray(data_test["Precipitation"]).astype(np.float32)
print(model.predict(x_test))
print(y_test)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[0.  0.  0.  0.  0.  0.  0.  0.  0.  2.5]


In [45]:
y_test = np.asarray(data["Precipitation"][1000010:1000012]).astype(np.float32)

x_test = np.asarray(df["Date"][1000010:1000012]).astype(np.float32)
print(x_test)
print(y_test)

[1.1217312e+18 1.1218176e+18]
[]


In [22]:
#test model 

model.evaluate(x_test, y_test, batch_size=32) 

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:923: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: ignored